In [1]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# Load CSV files into DataFrames
food_category_df = pd.read_csv('Data/food_category.csv')
food_df = pd.read_csv('Data/food.csv')
food_nutrient_df = pd.read_csv('Data/food_nutrient.csv')
nutrient_df = pd.read_csv('Data/nutrient.csv')

# Define the food categories you are interested in
specified_categories = ["Vegetables and Vegetable Products", 
                        "Legumes and Legume Products", 
                        "Spices and Herbs", 
                        "Cereal Grains and Pasta",
                        "Fats and Oils"]

# Filter the food_category DataFrame to include only the specified categories
filtered_food_category_df = food_category_df[food_category_df['description'].isin(specified_categories)]

# Merge DataFrames based on common keys
# Merge food_category and food on 'id' and 'food_category_id'
merged_food_category_food = pd.merge(filtered_food_category_df, food_df, left_on='id', right_on='food_category_id')

# Merge food and food_nutrient on 'fdc_id'
merged_food_nutrient = pd.merge(food_df, food_nutrient_df, on='fdc_id')

# Merge nutrient and food_nutrient on 'id' and 'nutrient_id'
merged_nutrient_food_nutrient = pd.merge(nutrient_df, food_nutrient_df, left_on='id', right_on='nutrient_id')

# Display first few rows of each merged DataFrame for verification
print("Merged food_category and food:")
print(merged_food_category_food.head())
print("\nMerged food and food_nutrient:")
print(merged_food_nutrient.head())
print("\nMerged nutrient and food_nutrient:")
print(merged_nutrient_food_nutrient.head())


Merged food_category and food:
   id  code     description_x  fdc_id       data_type  \
0   2   200  Spices and Herbs  170917  sr_legacy_food   
1   2   200  Spices and Herbs  170918  sr_legacy_food   
2   2   200  Spices and Herbs  170919  sr_legacy_food   
3   2   200  Spices and Herbs  170920  sr_legacy_food   
4   2   200  Spices and Herbs  170921  sr_legacy_food   

                   description_y  food_category_id publication_date  
0               Spices, bay leaf                 2       2019-04-01  
1           Spices, caraway seed                 2       2019-04-01  
2               Spices, cardamom                 2       2019-04-01  
3            Spices, celery seed                 2       2019-04-01  
4  Spices, coriander leaf, dried                 2       2019-04-01  

Merged food and food_nutrient:
   fdc_id       data_type                                        description  \
0  167512  sr_legacy_food  Pillsbury Golden Layer Buttermilk Biscuits, Ar...   
1  167512  sr_

In [2]:
# Step 1: Find the nutrient ID for "PUFA 18:3 n-3 c,c,c (ALA)"
nutrient_id = nutrient_df[nutrient_df['name'] == 'PUFA 18:3 n-3 c,c,c (ALA)']['id'].values
if len(nutrient_id) > 0:
    nutrient_id = nutrient_id[0]
else:
    nutrient_id = None

# Step 2: Filter the food_nutrient DataFrame for records where the nutrient value is greater than 0
filtered_food_nutrient_df = food_nutrient_df[(food_nutrient_df['nutrient_id'] == nutrient_id) & (food_nutrient_df['amount'] > 0)]

# Step 3: Merge the filtered food_nutrient DataFrame with the food DataFrame
result_df = pd.merge(filtered_food_nutrient_df, food_df, on='fdc_id')[['fdc_id', 'description', 'amount']]

# Display the result
print(result_df)


      fdc_id                                        description  amount
0     167523  Pie crust, deep dish, frozen, unbaked, made wi...   0.115
1     167525                               Tostada shells, corn   0.124
2     167526  Bread, salvadoran sweet cheese (quesadilla sal...   0.489
3     167527    Bread, pound cake type, pan de torta salvadoran   1.158
4     167528       Pastry, Pastelitos de Guava (guava pastries)   0.163
...      ...                                                ...     ...
1917  175245  Beans, kidney, red, mature seeds, canned, soli...   0.040
1918  175260  Lamb, New Zealand, imported, rack - partly fre...   0.113
1919  175261  Lamb, New Zealand, imported, rack - partly fre...   0.133
1920  175262  Lamb, New Zealand, imported, square-cut should...   0.256
1921  175264  Lamb, New Zealand, imported, square-cut should...   0.140

[1922 rows x 3 columns]


In [3]:
# load DataFrames and create the result_df goes here
# Interactive widgets
amount_slider = widgets.FloatSlider(
    value=0,
    min=0,
    max=result_df['amount'].max(),
    step=0.01,
    description='Amount:',
    continuous_update=False
)

# Function to update plot
def update_plot(amount):
    filtered_df = result_df[result_df['amount'] > amount]
    plt.figure(figsize=(10, 6))
    plt.barh(filtered_df['description'][:10], filtered_df['amount'][:10])
    plt.xlabel('Amount')
    plt.ylabel('Food Description')
    plt.title('Top 10 Foods based on PUFA 18:3 n-3 c,c,c (ALA)')
    plt.show()

# Interactive display
widgets.interactive(update_plot, amount=amount_slider)


interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='Amount:', max=53.368, step=…

In [6]:
# Function to create a comparison table for given fdc_ids
def create_comparison_table(fdc_ids):
    # Filter the merged_nutrient_food_nutrient DataFrame to only include the specified fdc_ids
    filtered_nutrients = merged_nutrient_food_nutrient[merged_nutrient_food_nutrient['fdc_id'].isin(fdc_ids)]
    
    # Convert non-numeric values to zero
    filtered_nutrients['amount'] = pd.to_numeric(filtered_nutrients['amount'], errors='coerce').fillna(0)
    
    # Pivot the DataFrame to get a comparison table
    comparison_df = pd.pivot_table(filtered_nutrients, values='amount', index=['name'], columns=['fdc_id'])
    
    # Reset the index for better formatting
    comparison_df.reset_index(inplace=True)
    
    # Apply conditional formatting to highlight the maximum value in each row
    styled_df = comparison_df.style.apply(highlight_max, axis=1)
    
    return styled_df


# Function to create a comparison table for given fdc_ids
def create_comparison_table(fdc_ids):
    # Filter the merged_nutrient_food_nutrient DataFrame to only include the specified fdc_ids
    filtered_nutrients = merged_nutrient_food_nutrient[merged_nutrient_food_nutrient['fdc_id'].isin(fdc_ids)]
    
    # Pivot the DataFrame to get a comparison table
    comparison_df = pd.pivot_table(filtered_nutrients, values='amount', index=['name'], columns=['fdc_id'])
    
    # Reset the index for better formatting
    comparison_df.reset_index(inplace=True)
    
    # Apply conditional formatting to highlight the maximum value in each row
    styled_df = comparison_df.style.apply(highlight_max, axis=1)
    
    return styled_df

# Function to highlight the maximum value in each row
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

# List of fdc_ids you want to compare (replace these with the actual fdc_ids)
fdc_ids_to_compare = [168428, 168427]  # Replace XXXX and YYYY with the actual FDC IDs

# Create the comparison table
styled_comparison_table = create_comparison_table(fdc_ids_to_compare)

# Display the styled DataFrame
display(styled_comparison_table)


TypeError: '>=' not supported between instances of 'str' and 'float'